In [5]:
# @title Install Dependencies
%%capture

from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/facebookresearch/detectron2.git
!python -m pip install -e detectron2
!pip install av

!pip install ninja
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing

!pip install -q mediapipe==0.10.0
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

!pip install opencv-python torchgeometry Pillow tqdm tensorboardX scikit-image scipy


# Import libraries
import shutil
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from google.colab import files
from PIL import Image, ImageOps
import glob
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
from google.colab.patches import cv2_imshow


shutil.copy('/content/drive/MyDrive/viton/densepose/mod_apply_net.py', '/content/detectron2/projects/DensePose/mod_apply_net.py')


MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green


def create_keypoints_json(rgb_image, body_detection_result, hands_detection_result, image_name):
  pose_landmarks_list = body_detection_result.pose_landmarks
  hand_landmarks_list = hands_detection_result.hand_landmarks
  handedness_list = hands_detection_result.handedness

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    kp_1_x = ((pose_landmarks[11].x * rgb_image.shape[1]) + (pose_landmarks[12].x * rgb_image.shape[1])) / 2
    kp_1_y = ((pose_landmarks[11].y * rgb_image.shape[0]) + (pose_landmarks[12].y * rgb_image.shape[0])) / 2
    kp_1_v = (pose_landmarks[11].visibility + pose_landmarks[12].visibility) / 2

    kp_8_x = ((pose_landmarks[23].x * rgb_image.shape[1]) + (pose_landmarks[24].x * rgb_image.shape[1])) / 2
    kp_8_y = ((pose_landmarks[23].y * rgb_image.shape[0]) + (pose_landmarks[24].y * rgb_image.shape[0])) / 2
    kp_8_v = (pose_landmarks[23].visibility + pose_landmarks[24].visibility) / 2

    pose_keypoints_2d = [
    pose_landmarks[0].x * rgb_image.shape[1], pose_landmarks[0].y * rgb_image.shape[0], pose_landmarks[0].visibility,
    kp_1_x, kp_1_y, kp_1_v,
    pose_landmarks[12].x * rgb_image.shape[1], pose_landmarks[12].y * rgb_image.shape[0], pose_landmarks[12].visibility,
    pose_landmarks[14].x * rgb_image.shape[1], pose_landmarks[14].y * rgb_image.shape[0], pose_landmarks[14].visibility,
    pose_landmarks[16].x * rgb_image.shape[1], pose_landmarks[16].y * rgb_image.shape[0], pose_landmarks[16].visibility,
    pose_landmarks[11].x * rgb_image.shape[1], pose_landmarks[11].y * rgb_image.shape[0], pose_landmarks[11].visibility,
    pose_landmarks[13].x * rgb_image.shape[1], pose_landmarks[13].y * rgb_image.shape[0], pose_landmarks[13].visibility,
    pose_landmarks[15].x * rgb_image.shape[1], pose_landmarks[15].y * rgb_image.shape[0], pose_landmarks[15].visibility,
    kp_8_x, kp_8_y, kp_8_v,
    pose_landmarks[24].x * rgb_image.shape[1], pose_landmarks[24].y * rgb_image.shape[0], pose_landmarks[24].visibility,
    pose_landmarks[26].x * rgb_image.shape[1], pose_landmarks[26].y * rgb_image.shape[0], pose_landmarks[26].visibility,
    pose_landmarks[28].x * rgb_image.shape[1], pose_landmarks[28].y * rgb_image.shape[0], pose_landmarks[28].visibility,
    pose_landmarks[23].x * rgb_image.shape[1], pose_landmarks[23].y * rgb_image.shape[0], pose_landmarks[23].visibility,
    pose_landmarks[25].x * rgb_image.shape[1], pose_landmarks[25].y * rgb_image.shape[0], pose_landmarks[25].visibility,
    pose_landmarks[27].x * rgb_image.shape[1], pose_landmarks[27].y * rgb_image.shape[0], pose_landmarks[27].visibility,
    pose_landmarks[5].x * rgb_image.shape[1], pose_landmarks[5].y * rgb_image.shape[0], pose_landmarks[5].visibility,
    pose_landmarks[2].x * rgb_image.shape[1], pose_landmarks[2].y * rgb_image.shape[0], pose_landmarks[2].visibility,
    pose_landmarks[8].x * rgb_image.shape[1], pose_landmarks[8].y * rgb_image.shape[0], pose_landmarks[8].visibility,
    pose_landmarks[7].x * rgb_image.shape[1], pose_landmarks[7].y * rgb_image.shape[0], pose_landmarks[7].visibility,
    pose_landmarks[29].x * rgb_image.shape[1], pose_landmarks[29].y * rgb_image.shape[0], pose_landmarks[29].visibility,
    pose_landmarks[31].x * rgb_image.shape[1], pose_landmarks[31].y * rgb_image.shape[0], pose_landmarks[31].visibility,
    pose_landmarks[27].x * rgb_image.shape[1], pose_landmarks[27].y * rgb_image.shape[0], pose_landmarks[27].visibility,
    pose_landmarks[30].x * rgb_image.shape[1], pose_landmarks[30].y * rgb_image.shape[0], pose_landmarks[30].visibility,
    pose_landmarks[32].x * rgb_image.shape[1], pose_landmarks[32].y * rgb_image.shape[0], pose_landmarks[32].visibility,
    pose_landmarks[28].x * rgb_image.shape[1], pose_landmarks[28].y * rgb_image.shape[0], pose_landmarks[28].visibility
    ]


  # Loop through the detected hands to visualize.

  if len(hand_landmarks_list) != 0:

    for idx in range(len(hand_landmarks_list)):
      hand_landmarks = hand_landmarks_list[idx]
      handedness = handedness_list[idx]

      hand_right_keypoints_2d = []
      hand_left_keypoints_2d = []
      if handedness[0].category_name == 'Right':
        for lmk in hand_landmarks:
          hand_right_keypoints_2d.append(lmk.x * rgb_image.shape[1])
          hand_right_keypoints_2d.append(lmk.y * rgb_image.shape[0])
          hand_right_keypoints_2d.append(lmk.visibility)
      if handedness[0].category_name == 'Left':
        for lmk in hand_landmarks:
          hand_left_keypoints_2d.append(lmk.x * rgb_image.shape[1])
          hand_left_keypoints_2d.append(lmk.y * rgb_image.shape[0])
          hand_left_keypoints_2d.append(lmk.visibility)

  else:
    hand_right_keypoints_2d = []
    hand_left_keypoints_2d = []



  json_dict = {
        "version":1.3,
        "people":[
            {
                "person_id":[-1],
                "pose_keypoints_2d": pose_keypoints_2d,
                "face_keypoints_2d":[],
                "hand_left_keypoints_2d": hand_left_keypoints_2d,
                "hand_right_keypoints_2d": hand_right_keypoints_2d,
                "pose_keypoints_3d":[],
                "face_keypoints_3d":[],
                "hand_left_keypoints_3d":[],
                "hand_right_keypoints_3d":[]
            }
        ]
  }

  json_obj = json.dumps(json_dict)

  with open(f'/content/drive/MyDrive/viton/HR/data/test/openpose_json/{image_name}_keypoints.json', 'w') as f:
    f.write(json_obj)

In [ ]:
# @title Upload Person Image
#Please select images which you want to upload
shutil.rmtree('/content/drive/MyDrive/viton/HR/data/test/image')

!mkdir /content/drive/MyDrive/viton/HR/data/test/image
%cd /content/drive/MyDrive/viton/HR/data/test/image

print("Upload Person Image: ")
uploaded = files.upload()
image_name = list(uploaded.keys())[0][:-4]


image = Image.open(f'/content/drive/MyDrive/viton/HR/data/test/image/{image_name}.jpg')
image = ImageOps.fit(image, (768, 1024))
image.save(f'/content/drive/MyDrive/viton/HR/data/test/image/{image_name}.jpg')

%cd /content

cloth_dir = '/content/drive/MyDrive/viton/HR/data/test/cloth'
cloth_ids = os.listdir(cloth_dir)
print(f"Choose Cloth id from: {cloth_ids}")
cloth_id = input()


with open('/content/drive/MyDrive/viton/HR/data/test_pairs.txt', 'w')  as f:
  f.write(f'{image_name}.jpg {cloth_id}.jpg')

In [ ]:
# @title Apply Virtual-Try-On
%cd /content/detectron2/projects/DensePose
!python mod_apply_net.py show /content/drive/MyDrive/viton/densepose/configs/densepose_rcnn_R_50_FPN_s1x.yaml /content/drive/MyDrive/viton/densepose/models/model_final_162be9.pkl /content/drive/MyDrive/viton/HR/data/test/image dp_segm --output output.jpg


dp_img = Image.open(glob.glob('/content/detectron2/projects/DensePose/*.jpg')[0])
dp_img.save(f'/content/drive/MyDrive/viton/HR/data/test/image-densepose/{image_name}.jpg')

op_img = Image.open('/content/drive/MyDrive/viton/HR/data/test/openpose_img/sample_rendered.png')
op_img.save(f'/content/drive/MyDrive/viton/HR/data/test/openpose_img/{image_name}_rendered.png')

%cd /content/Self-Correction-Human-Parsing
!python3 simple_extractor.py --dataset 'lip' --model-restore /content/drive/MyDrive/viton/schp/model/exp-schp-201908261155-lip.pth --input-dir /content/drive/MyDrive/viton/HR/data/test/image --output-dir /content/drive/MyDrive/viton/HR/data/test/image-parse-v3
%cd /content

shutil.copy(f'/content/drive/MyDrive/viton/HR/data/test/cloth/{cloth_id}.jpg', f'/content/drive/MyDrive/viton/HR/data/test/cloth/{image_name}.jpg')
shutil.copy(f'/content/drive/MyDrive/viton/HR/data/test/cloth-mask/{cloth_id}.jpg', f'/content/drive/MyDrive/viton/HR/data/test/cloth-mask/{image_name}.jpg')



image = mp.Image.create_from_file(f"/content/drive/MyDrive/viton/HR/data/test/image/{image_name}.jpg")

# For Body Landmarks
# STEP 2: Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# STEP 4: Detect pose landmarks from the input image.
body_detection_result = detector.detect(image)

# For Hands_landmarks
# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

# STEP 4: Detect hand landmarks from the input image.
hands_detection_result = detector.detect(image)

# STEP 5: Process the detection result. In this case, visualize it.
create_keypoints_json(image.numpy_view(), body_detection_result, hands_detection_result, image_name)


%cd /content/drive/MyDrive/viton/HR
!python get_parse_agnostic.py --data_path data/test --output_path data/test/image-parse-agnostic-v3.2
!python test_generator.py --occlusion --cuda True --test_name test_name --gpu_ids 0 --datasetting unpaired --dataroot data --data_list test_pairs.txt


person = cv2.imread(f"/content/drive/MyDrive/viton/HR/data/test/image/{image_name}.jpg")
img = cv2.imread(f"/content/drive/MyDrive/viton/HR/Output/{image_name}_{cloth_id}.png")
cloth = cv2.imread(f"/content/drive/MyDrive/viton/HR/data/test/cloth/{cloth_id}.jpg")
final_img = np.concatenate([person, cloth, img], axis=1)

cv2_imshow(final_img)